# Import the necessary libraries

In [24]:
import torch

from models.emage_audio import EmageVAEConv, EmageVQVAEConv, EmageVQModel
from models.emage_audio import EmageAudioModel
from peft import get_peft_model, LoraConfig, TaskType

# Load the pre-trained models

In [25]:
pretrained_path = "H-Liu1997/emage_audio"

face_model_path = "emage_vq/face"
upper_model_path = "emage_vq/upper"
lower_model_path = "emage_vq/lower"
hands_model_path = "emage_vq/hands"
global_model_path = "emage_vq/global"

In [26]:
device = torch.device(device='cuda:0')

face_motion_vq = EmageVQVAEConv.from_pretrained(pretrained_path, subfolder=face_model_path)
upper_motion_vq = EmageVQVAEConv.from_pretrained(pretrained_path, subfolder=upper_model_path)
lower_motion_vq = EmageVQVAEConv.from_pretrained(pretrained_path, subfolder=lower_model_path)
hands_motion_vq = EmageVQVAEConv.from_pretrained(pretrained_path, subfolder=hands_model_path)
global_motion_ae = EmageVAEConv.from_pretrained(pretrained_path, subfolder=global_model_path)
motion_vq = EmageVQModel(
    face_model=face_motion_vq, upper_model=upper_motion_vq,
    lower_model=lower_motion_vq, hands_model=hands_motion_vq,
    global_model=global_motion_ae)

audio_model = EmageAudioModel.from_pretrained(pretrained_path)

print(motion_vq)

print(audio_model)

EmageVQModel(
  (vq_model_face): EmageVQVAEConv(
    (encoder): VQEncoderV5(
      (main): Sequential(
        (0): Conv1d(106, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): LeakyReLU(negative_slope=0.2, inplace=True)
        (2): ResBlock(
          (model): Sequential(
            (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): LeakyReLU(negative_slope=0.2, inplace=True)
            (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          )
        )
        (3): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): LeakyReLU(negative_slope=0.2, inplace=True)
        (5): ResBlock(
          (model): Sequential(
            (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): LeakyReLU(negative_slope=0.2, inplace=True)
            (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          )
        )
      )
    )
    (quantizer): Quantizer(
 

In [27]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    target_modules=['embedding'],
    lora_alpha=32,
    lora_dropout=0.2
)

motion_vq = get_peft_model(motion_vq, lora_config)

print(motion_vq)

AttributeError: 'EmageVQModel' object has no attribute 'prepare_inputs_for_generation'